# RecDP LLM - Downsize public dataset

This notebook shows how to use RecDP tools and pipeline to downsize a public fine tuning dataset and calculate kinds of evaluation scores.

# Get Started

## 1. Install pyrecdp and dependencies

In [1]:
! DEBIAN_FRONTEND=noninteractive apt-get install -qq -y openjdk-8-jre
! pip install 'git+https://github.com/intel/e2eAIOK.git#egg=pyrecdp&subdirectory=RecDP'

  Cloning https://github.com/intel/e2eAIOK.git to /tmp/pip-install-p1dgcloc/pyrecdp_c73eeefeffb64f78a7992379abd3e063
  Running command git clone --filter=blob:none --quiet https://github.com/intel/e2eAIOK.git /tmp/pip-install-p1dgcloc/pyrecdp_c73eeefeffb64f78a7992379abd3e063
  Resolved https://github.com/intel/e2eAIOK.git to commit f0ce056e5a435b4224b93cb8efb7fa9b678dea69
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


  Created wheel for pyrecdp: filename=pyrecdp-1.6.1-py3-none-any.whl size=375563 sha256=81ba14d8d440941fa8fca2c16571478715f375e568e4e5f816f363d6dc1515a9
  Stored in directory: /tmp/pip-ephem-wheel-cache-il0wu9c4/wheels/1a/a7/22/3c0128609422765ab16fdd4095b9a9b426e5ae9de5c15cf70c
Successfully built pyrecdp
  Attempting uninstall: pyrecdp
    Found existing installation: pyrecdp 1.6.1
    Uninstalling pyrecdp-1.6.1:
      Successfully uninstalled pyrecdp-1.6.1


## 2. Prepare your data

In [2]:
%mkdir -p /content/test_data
!wget -P /content/test_data https://raw.githubusercontent.com/intel/e2eAIOK/main/RecDP/tests/data/alpaca/alpaca_data_50.jsonl

--2023-11-28 10:17:05--  https://raw.githubusercontent.com/intel/e2eAIOK/main/RecDP/tests/data/alpaca/alpaca_data_50.jsonl
Resolving child-prc.intel.com (child-prc.intel.com)... 10.239.120.55
Connecting to child-prc.intel.com (child-prc.intel.com)|10.239.120.55|:913... connected.
Proxy request sent, awaiting response... 200 OK
Length: 25027 (24K) [text/plain]
Saving to: ‘/content/test_data/alpaca_data_50.jsonl’

alpaca_data_50.json 100%[===================>]  24.44K   126KB/s    in 0.2s    

2023-11-28 10:17:07 (126 KB/s) - ‘/content/test_data/alpaca_data_50.jsonl’ saved [25027/25027]



## 3. Downsize dataset

### 3.1 Pipeline on Spark

In [7]:
from pyrecdp.LLM import TextPipeline, ResumableTextPipeline
from pyrecdp.primitives.operations import *

JAVA_HOME is not set, use default value of /usr/lib/jvm/java-8-openjdk-amd64/


/opt/conda/envs/llm_data/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


['psutil', 'tqdm', 'pyyaml', 'pandas', 'pyarrow', 'transformers', 'graphviz', 'requests', 'distro', 'pyspark==3.4.0', 'matplotlib', 'datasketch==1.5.9', 'ftfy==6.1.1', 'jsonlines==3.1.0', 'networkit==10.1', 'nltk==3.8.1', 'regex==2023.6.3', 'scipy==1.10.1', 'typer>=0.6.1', 'phonenumbers', 'fasttext==0.9.2', 'wget==3.2', 'alt-profanity-check==1.3.0', 'huggingface-hub', 'loguru==0.7.2', 'tabulate==0.9.0', 'sentencepiece', 'selectolax', 'spacy', 'torch', 'Faker', 'ray==2.7.1', 'loguru', 'detoxify', 'emoji==2.2.0', 'kenlm', 'rouge-score']


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Requirement already satisfied: alt-profanity-check==1.3.0 in /opt/conda/envs/llm_data/lib/python3.9/site-packages (1.3.0)

...

Requirement already satisfied: joblib in /opt/conda/envs/llm_data/lib/python3.9/site-packages (from nltk->rouge-score) (1.3.2)

Requirement already satisfied: regex>=2021.8.3 in /opt/conda/envs/llm_data/lib/python3.9/site-packages (from nltk->rouge-score) (2023.6.3)

Requirement already satisfied: tqdm in /opt/conda/envs/llm_data/lib/python3.9/site-packages (from nltk->rouge-score) (4.66.1)

WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv


/opt/conda/envs/llm_data/lib/python3.9/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [4]:
import os
spark_pipeline = ResumableTextPipeline()
spark_pipeline.enable_statistics()
out_dir = "ResumableTextPipeline_output-spark"
ops = [
    JsonlReader("/content/test_data/alpaca_data_50.jsonl"),
    TextPrompt(dataset_name="alpaca", prompt_name="causal_llm_1"),
    RandomSelect(fraction=0.3),
    TextToxicity(huggingface_config_path="/root/.cache/huggingface/hub/models--xlm-roberta-base"),
    TextDiversityIndicate(out_dir=out_dir, language="en", first_sent=False),
    TextQualityScorer(model="gpt3"),
    RougeScoreDedup(max_ratio=0.7, batch_size=10,score_store_path=os.path.join(out_dir,'RougeScorefiltered.parquet')),
    ParquetWriter(out_dir)
]
spark_pipeline.add_operations(ops)
ret = spark_pipeline.execute()

2023-11-28 10:17:42.527 | WARNING  | pyrecdp.LLM.TextPipeline:enable_statistics:219 - Enabling this option will result in a decrease in execution speed
[DatasetReader, PerfileSourcedJsonlReader, TextPrompt, RandomSelect, TextToxicity, TextDiversityIndicate, TextQualityScorer, RougeScoreDedup, PerfileParquetWriter]
Will assign 36 cores and 206263 M memory for spark


23/11/28 10:17:44 WARN Utils: Your hostname, sr533 resolves to a loopback address: 127.0.1.1; using 10.0.2.133 instead (on interface eno0)
23/11/28 10:17:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/28 10:17:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/28 10:17:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


per core memory size is 5.595 GB and shuffle_disk maximum capacity is 8589934592.000 GB
execute with spark for global tasks started ...
DatasetReader
2023-11-28 10:17:46.910 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:285 - DatasetReader: A total of 0 rows of data were processed, using 0 seconds, with 0 rows modified or removed, 0 rows of data remaining.
execute with spark for global tasks took 0.002241840586066246 sec
PerfileSourcedJsonlReader


ResumableTextPipeline, current on alpaca_data_50.jsonl:   0%|          | 0/1 [00:00<?, ?it/s]

alpaca_data_50.jsonl
TextPrompt


...


0it [00:00, ?it/s]

generate_connected_components => duplicates took 0.01515495590865612 sec


2023-11-28 10:19:22.109 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:285 - TextPrompt: A total of 0 rows of data were processed, using 0 seconds, with 0 rows modified or removed, 0 rows of data remaining.
2023-11-28 10:19:22.111 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:285 - RandomSelect: A total of 0 rows of data were processed, using 0 seconds, with 0 rows modified or removed, 0 rows of data remaining.
2023-11-28 10:19:22.114 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:281 - TextToxicity: A total of 11 rows of data were processed, using 54.51850247383118 seconds, Get max toxicity 0.0010136101627722383, Get min toxicity 0.0001716656406642869, Get average toxicity 0.00036636507800060576,Get the std of toxicity 0.0002271174606949808
2023-11-28 10:19:22.122 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:281 - TextDiversityIndicate: A total of 11 rows of data were processed, using 13.499625205993652 seconds, Get max diversity types 11, Get average diversity types 1.625,

ResumableTextPipeline, current on alpaca_data_50.jsonl: 100%|██████████| 1/1 [01:28<00:00, 88.99s/it]

2023-11-28 10:19:22.133 | INFO     | pyrecdp.LLM.TextPipeline:execute:425 - Completed! ResumableTextPipeline will not return dataset, please check ResumableTextPipeline_output-spark for verification.


### 3.2 Check the result and score

In [5]:
!ls ResumableTextPipeline_output-spark

IOStream.flush timed out
IOStream.flush timed out
RougeScoreDedup-statistics	  TextToxicity-statistics  pipeline
RougeScorefiltered.parquet	  alpaca_data_50.jsonl	   pipeline.json
TextDiversityIndicate-statistics  diversity.csv		   pipeline.log
TextQualityScorer-statistics	  diversity.md		   status.log


In [5]:
import json
import pandas as pd

print('The Rouge score: ', json.load(open('ResumableTextPipeline_output-spark/RougeScoreDedup-statistics', 'r')))
print('The toxicity: ', json.load(open('ResumableTextPipeline_output-spark/TextToxicity-statistics', 'r')))
print('The diversity intricate: ', json.load(open('ResumableTextPipeline_output-spark/TextDiversityIndicate-statistics', 'r')))
print('The quality score :', json.load(open('ResumableTextPipeline_output-spark/TextQualityScorer-statistics', 'r')))


origin_dataset_length = len(pd.read_json("/content/test_data/alpaca_data_50.jsonl", lines=True))
downsized_dataset_length = len(pd.read_parquet('ResumableTextPipeline_output-spark/alpaca_data_50.jsonl'))

print(f'The original dataset length: {origin_dataset_length}, the processed dataset length: {downsized_dataset_length}')

The Rouge score:  {'dup_num': 0, 'dup_ratio': 0.0}
The toxicity:  {'min': 0.0001716656406642869, 'max': 0.0010136101627722383, 'mean': 0.00036636507800060576, 'std': 0.0002271174606949808}
The diversity intricate:  {'max': 11, 'mean': 1.625, 'std': 2.5}
The quality score : {'mean': 0.9509534303108631}
The original dataset length: 50, the processed dataset length: 11


### 3.3 Pipeline on Ray

In [8]:
ray_pipeline = ResumableTextPipeline()
ray_pipeline.enable_statistics()
out_dir = "ResumableTextPipeline_output-ray"
ops = [
    JsonlReader("/content/test_data/alpaca_data_50.jsonl"),
    TextPrompt(dataset_name="alpaca", prompt_name="causal_llm_1"),
    RandomSelect(fraction=0.3),
    TextPerplexityScore(),
    ParquetWriter(out_dir)
]
ray_pipeline.add_operations(ops)
ret = ray_pipeline.execute()

2023-11-28 10:23:23.009 | WARNING  | pyrecdp.LLM.TextPipeline:enable_statistics:219 - Enabling this option will result in a decrease in execution speed
2023-11-28 10:23:23.012 | INFO     | pyrecdp.core.model_utils:prepare_sentencepiece_model:108 - Loading sentencepiece model...
2023-11-28 10:23:23.092 | INFO     | pyrecdp.core.model_utils:prepare_kenlm_model:126 - Loading kenlm language model...
[DatasetReader, PerfileSourcedJsonlReader, TextPrompt, RandomSelect, TextPerplexityScore, PerfileParquetWriter]
init ray with total mem of 162212234035


2023-11-28 10:23:27,532	WARNING services.py:1889 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67100672 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-11-28 10:23:28,621	INFO worker.py:1642 -- Started a local Ray instance.
2023-11-28 10:23:31,164	INFO read_api.py:406 -- To satisfy the requested parallelism of 144, each read task output is split into 144 smaller blocks.
2023-11-28 10:23:31,209	INFO streaming_executor.py:93 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadJSON->SplitBlocks(144)] -> TaskPoolMapOperator[Map(<lambda>)->Map(<lambda>)->MapBatches(<lambda>)]
2023-11-28 10:23:31,211	INFO streaming_executor.py:94 -- Execut

Running 0:   0%|          | 0/20736 [00:00<?, ?it/s]

ResumableTextPipeline, current on alpaca_data_50.jsonl:   0%|          | 0/1 [00:00<?, ?it/s]2023-11-28 10:23:33,498	INFO streaming_executor.py:93 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadJSON->SplitBlocks(144)] -> TaskPoolMapOperator[Map(<lambda>)->Map(<lambda>)->MapBatches(<lambda>)->Map(<lambda>)->MapBatches(process_batch)]
2023-11-28 10:23:33,499	INFO streaming_executor.py:94 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2023-11-28 10:23:33,500	INFO streaming_executor.py:96 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


Running 0:   0%|          | 0/20736 [00:00<?, ?it/s]

...

Running 0:   0%|          | 0/20736 [00:00<?, ?it/s]

(Map(<lambda>)->Map(<lambda>)->MapBatches(<lambda>)->Map(<lambda>)->MapBatches(process_batch)->Map(<lambda>)->Write pid=175868) Skipping writing empty dataset with UUID dc0b447b9d4a4287bc9cb17c6b4959b9 at ResumableTextPipeline_output-ray/alpaca_data_50.jsonl
(Map(<lambda>)->Map(<lambda>)->MapBatches(<lambda>)->Map(<lambda>)->MapBatches(process_batch) pid=175828) /opt/conda/envs/llm_data/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils. [repeated 71x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(Map(<lambda>)->Map(<lambda>)->MapBatches(<lambda>)->Map(<lambda>)->MapBatches(process_batch) pid=175828)   warnings.warn("Setuptools is replacing distutils.") [repeated 71x across cluster]


2023-11-28 10:24:06.379 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:285 - TextPrompt: A total of 0 rows of data were processed, using 0 seconds, with 0 rows modified or removed, 0 rows of data remaining.
2023-11-28 10:24:06.381 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:285 - RandomSelect: A total of 0 rows of data were processed, using 0 seconds, with 0 rows modified or removed, 0 rows of data remaining.
2023-11-28 10:24:06.382 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:281 - TextPerplexityScore: A total of 12 rows of data were processed, using 11.1940016746521 seconds, Get max perplexity 1284.5, Get min perplexity 158.0, Get average perplexity 444.9384615384616,Get the std of perplexity 236.0717117838882
2023-11-28 10:24:06.385 | INFO     | pyrecdp.LLM.TextPipeline:op_summary:285 - PerfileParquetWriter: A total of 0 rows of data were processed, using 0 seconds, with 0 rows modified or removed, 0 rows of data remaining.


ResumableTextPipeline, current on alpaca_data_50.jsonl: 100%|██████████| 1/1 [00:32<00:00, 32.91s/it]

2023-11-28 10:24:06.389 | INFO     | pyrecdp.LLM.TextPipeline:execute:425 - Completed! ResumableTextPipeline will not return dataset, please check ResumableTextPipeline_output-ray for verification.


### 3.4 Check the result and score

In [9]:
!ls ResumableTextPipeline_output-ray

TextPerplexityScore-statistics	pipeline       pipeline.log
alpaca_data_50.jsonl		pipeline.json  status.log


In [11]:
import json
import pandas as pd

print('The Rouge score: ', json.load(open('ResumableTextPipeline_output-ray/TextPerplexityScore-statistics', 'r')))

origin_dataset_length = len(pd.read_json("/content/test_data/alpaca_data_50.jsonl", lines=True))
downsized_dataset_length = len(pd.read_parquet('ResumableTextPipeline_output-ray/alpaca_data_50.jsonl'))

print(f'The original dataset length: {origin_dataset_length}, the processed dataset length: {downsized_dataset_length}')

The Rouge score:  {'min': 158.0, 'max': 1284.5, 'mean': 444.9384615384616, 'std': 236.0717117838882}
The original dataset length: 50, the processed dataset length: 19
